In [108]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine!
Combined_df = pd.merge(mouse_metadata, study_results, on = 'Mouse ID')

# Display the data table for preview
print(Combined_df)

     Mouse ID Drug Regimen   Sex  Age_months  Weight (g)  Timepoint  \
0        k403     Ramicane  Male          21          16          0   
1        k403     Ramicane  Male          21          16          5   
2        k403     Ramicane  Male          21          16         10   
3        k403     Ramicane  Male          21          16         15   
4        k403     Ramicane  Male          21          16         20   
...       ...          ...   ...         ...         ...        ...   
1888     z969     Naftisol  Male           9          30         25   
1889     z969     Naftisol  Male           9          30         30   
1890     z969     Naftisol  Male           9          30         35   
1891     z969     Naftisol  Male           9          30         40   
1892     z969     Naftisol  Male           9          30         45   

      Tumor Volume (mm3)  Metastatic Sites  
0              45.000000                 0  
1              38.825898                 0  
2           

In [109]:
# Checking the number of mice.
Numberuniquemice = len(pd.unique(Combined_df['Mouse ID']))
Numberuniquemice

249

In [114]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint.
Duplicate_ID= []
#Find the Duplicates as determined by Mouse ID and Timepoint by putting the ID's in Duplicate_ID
for i in range(len(Combined_df)-1):
        if Combined_df.iloc[i]['Timepoint'] == Combined_df.iloc[i+1]['Timepoint']:
            Duplicate_ID.append(Combined_df.iloc[i]['Mouse ID'])
#Change the Index
Reindex_df = Combined_df.set_index(['Mouse ID'])
#Drop any Mouse ID's matching Duplicate_ID's values
Reindex_df = Reindex_df.drop(index = Duplicate_ID)
Resetindex_df = Reindex_df.reset_index()

In [111]:
#Number of unique mice in cleaned data
Numberofcleanmice = len(pd.unique(Resetindex_df['Mouse ID']))
print(Numberofcleanmice)

236


In [126]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
Tumor_summary_statistics = Resetindex_df.groupby('Drug Regimen')['Tumor Volume (mm3)'].describe()
Tumor_summary_statistics


#Regimengroup_df = Resetindex_df.groupby(['Drug Regimen'])
#for key, item in Regimengroup_df:
    #print(Regimengroup_df.get_group(key), "\n\n")
# This method is the most straighforward, creating multiple series and putting them all together at the end.


,count,mean,std,min,25%,50%,75%,max
Drug Regimen,,,,,,,,
Capomulin,230.0,40.675741,4.994774,23.343598,37.685933,41.557809,45.000000,48.158209
Ceftamin,174.0,52.765682,6.231822,45.000000,47.362986,51.910178,57.080919,68.923185
Infubinol,178.0,52.884795,6.567243,36.321346,47.312353,51.820584,57.314444,72.226731
Ketapril,186.0,55.345698,8.255295,45.000000,48.392614,53.739593,60.899768,78.567014
Naftisol,184.0,54.432995,8.120017,45.000000,47.405316,52.544915,60.023500,76.668817
Placebo,180.0,54.083767,7.813538,45.000000,47.505237,52.316010,59.962344,73.212939
Propriva,147.0,52.370732,6.616855,45.000000,47.011050,50.712423,56.497951,72.455421
Ramicane,228.0,40.216745,4.846308,22.050126,36.674635,40.673236,45.000000,47.622816
Stelasyn,180.0,54.284444,7.700895,45.000000,48.149336,52.488221,58.792405,75.123690
